# Processing Enformer predictions 

This script takes the raw predictions made for  EXTRA-seq fragments embedded in the endogenous AXIN2 locus using Enformer and extract the scores for the UTR (expression) and the local activities at the enhancer in GM12878.

In [1]:
# for python
import gc
import itertools
import matplotlib.pyplot as plt
import numpy as np
import random

import pandas as pd
import rpy2.rinterface as rinterface
import rpy2.robjects as robjects
import tqdm
import seaborn as sns
import scipy.stats as stats

from itertools import compress
from Bio import motifs
from Bio.Seq import Seq #, IUPAC
from collections import Counter
from os import listdir
from os.path import join
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from scipy.stats import ks_2samp
from statistics import mean, median

%load_ext rpy2.ipython

%matplotlib inline
pandas2ri.activate()
plt.ioff()

In [41]:
%%R
library(Biostrings)
library("BSgenome.Hsapiens.UCSC.hg19")
library(tidyverse)
library(dplyr)
library(reshape2)


# general functions
makeTr <- function(someColor, alpha=100) scales::alpha(someColor, alpha/100)


In [ ]:
library(stringi)
library(rtracklayer)
library(ggplot2)
library(patchwork)
library(gplots)
library(RColorBrewer)
library(plotly)
library("wesanderson",lib.loc="/home/kribelba/R_libs/")
library(GenomicFeatures)
library("TxDb.Hsapiens.UCSC.hg19.knownGene")
library('AnnotationHub')
library(GO.db)
library(org.Hs.eg.db)
library(ChIPseeker)
library(ggpubr)

library(viridis)
library(DESeq2)
library(Rsubread)
library("tximport")
library("readr")
library(biomaRt)
library(ggbeeswarm)


### Define sequence features

In [3]:
%%R
loxP = 'ATAACTTCGTATAATGTATGCTATACGAAGTTAT'
lox22 = 'ATAACTTCGTATAGGATACTTTATACGAAGTTAT'
# BC of length 10bp for 4.9, 12bp for 5.1,5.7,5.8 libs, 14bp for 3.8,3.98, 4.88 libs
BC10 = paste(rep('N',10),collapse="")
BC12 = paste(rep('N',12),collapse="")
BC14 = paste(rep('N',14),collapse="")

#region inbetween the UTR and the enhancer
ibtw_wt = 'GGTCTGTCCGTCATGCTGAGGGGTTATTTTTATTTCCCGGCTCTCGGGCTGTTACTGAGTTGCCAGGACCTTATCAAAGCGCAGCCGGCTCCAGCCGACTCCCCCGGGCCAGGCTCGCGGAGCCAGTGATCCCGCCGCGCCAATCACAGCCGCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGACAGCGCCCCGCGCCCGCCCCGCCCCGCCCCGGCCCGCCCCCGCCGCACGCTCCGCTCCCGCGTTAACCCTTCCCCGGCCTCTCCTCCGCCCCCTGCCAGCCCCCCGCGGCCCCGAGGGCATTCTCAGCCCGAGTTCTGATTTCTTCTTTTAAGTTCTCCGGCCTCTTTGCTCCCGCCTCCTCCCCCTGCCTGCCCTCGCTCTCCCCGGCTCTGGAATGCAACAGTTTCTGGTAGCATTATGGCCATCGCAAGAACTGCAAGCAAGCAGATTTTTTTTTTTCTATCATCAAACGCTTTTCTACCTCTATTCAAGTTTCCCTGGGAATTTGGGCTTGGAGGACTTTATAAAGGGAGTTCCGGCCCTGGGGTTGGCTGGCCCGCTGTCATAGGGCGGACTCGGGCCCAGCCGCTGGAGGCGTGTGCATGCCTGTGCGCGCTTGTGCACGTGCATCTGAGTTCGGGTAAATATAGGACTGCAGTTGGTGGATGAATTGCGTTTGTGCATGTGTGAAAGTGGGGTGTGAATCCGCCAGTATAGGCCTGAGTGCTAACACCAGTTAGGAGGCATACCTCAGTGCCCTGAAGACCTAGGTCCAGCTTCCTAACCTGCTGGCCATAAGACCCTCGTAGCGGCACACATTGGCCATCTCTCCCAAACAACTGCATACCCGTTACTCTTCCAAAGTCCCCTCTTCTGCCATATGTTACATATTAAACCGATAGGAATATCTTTTTTCCCACTCCAAACGACAAGGCATGCATGCCGTTTTAATGTATCGAAAACTCCTCACCCCATTGTCACCTTTGAATTTCTCCCACATTTGCCCAAAAACGTTTATAAGGCACACACAACCTTCTCTTCTCCGTTAAAATGAGAAAATATGTCACAAACCCAGAGCATTGTGTATTTATTCTGCTGCTGCTATAGAATGATTCCTTATGTTATGAGCGAATAATTAACTTTTCTTCCTTTTAAAAAACAAAGGGCCGGGGACGGTGGCTCATGCCTGTAATCCCAGCATTTTGGGAGGCCCATGCTGGAGGATCATCTGAGGTCAGGAGTTCGAGACCAGCCTGCCTAATATGGCGAAACCCCGTCTCTACTAAAAATACAAAAATCAGCCGGGTGTGGTGGCACGTGCCTGTAGTCCCAGCTACTCGGGAGGCTGAGGCAGGAGAATCGCTTGAACCTAGGAGACGGGGGTGGGGGGGTTGCAGTGAGCCAAGATGGCACCACTGCACGCCAGGCTGGGTGACAAAGCGAGACTCCCTCAAAAAAAAAAAAAAAAAACACACACACACACACACATAAAGGTGTGAACCAAAGAGATCCAAGAAGTGAAAGGCAGAGTCCTGGCCTCAGAGCTGTGCAAGGCTCTTACTCTCCAACTTAATGGATGACGAGCCCTTTTAAATGCGAGATGGCCTGTCATTAGGCCTTCAGAACTTTAAAACATGCAAGAATTGTGGCTCTATCTAGGTATCCATAGAAAAAGAGAAGGAAGAGGCTGGGTGTGGTGGCTCACCCGTGTAATCCCAGCACTTCGGGAGGCCAAGGCAGGCGGATCACCTGAGGTCAGGCATTCGAGACCAGCCTGGCCAACATAGTGAAACCCTGTCCCTACTAAAACTACAAATATTAGCCAGGCGTGGTGGGGGGCGCCTGTAGTCCCAGCTACTCGGGAGGCTGAGGCAGGAGAATCATTTAAACACAGGAGGTTAAGGTTGCAGTGAGCCTGGGCGACAGAGCAAGACTCCATCTCAAAAAAAAAAAAGAAAAGAAAAAAAGAGGAGGAAGATCCACCACCCATGATCTGCTGGAAAGGGGCAGGTGGCAGGACTGTGCGCCACCTGCCCTCAGCCTAAGGGACTGTGACAAGGGACTAGAAAGCTCTTAGACTTTCTAGTCTAAGACTCTGGACTCTGGCTCGTGGGTGCCATGACAGGTGGCCGCTCCTCCCCAAAACCTGCCTTTCGGTGCCCATGGTCTGCAGCTCCAGGC'

# specific to lib 5.1, 5.7 and 5.8 (synthetic and out-of-context enhancers
art_E_end = 'GCAGTCCAGTGTACCACGTTG' #primer site added upstream of synthetic fragement for amplification

# native Axin2 enhancer context
BS_left_flank = 'GAGAGAGAAAAATCAAA' #flanking sequence to the left (downstream) of MEF2/indel location
BS_right_flank = 'AATGTTCAGAAAAAAGAGG'#flanking sequence to the right (downstream) of MEF2/indel location

# sequence of ALT/REF genotyeps starting from inbetween region
alt = 'GGTCTGTCCGTCATGCTGAGGGGTTATTTTTATTTCCCGGCTCTCGGGCTGTTACTGAGTTGCCAGGACCTTATCAAAGCGCAGCCGGCTCCAGCCGACTCCCCCGGGCCAGGCTCGCGGAGCCAGTGATCCCGCCGCGCCAATCACAGCCGCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGACAGCGCCCCGCGCCCGCCCCGCCCCGCCCCGGCCCGCCCCCGCCGCACGCTCCGCTCCCGCGTTAACCCTTCCCCGGCCTCTCCTCCGCCCCCTGCCAGCCCCCCGCGGCCCCGAGGGCATTCTCAGCCCGAGTTCTGATTTCTTCTTTTAAGTTCTCCGGCCTCTTTGCTCCCGCCTCCTCCCCCTGCCTGCCCTCGCTCTCCCCGGCTCTGGAATGCAACAGTTTCTGGTAGCATTATGGCCATCGCAAGAACTGCAAGCAAGCAGATTTTTTTTTTTCTATCATCAAACGCTTTTCTACCTCTATTCAAGTTTCCCTGGGAATTTGGGCTTGGAGGACTTTATAAAGGGAGTTCCGGCCCTGGGGTTGGCTGGCCCGCTGTCATAGGGCGGACTCGGGCCCAGCCGCTGGAGGCGTGTGCATGCCTGTGCGCGCTTGTGCACGTGCATCTGAGTTCGGGTAAATATAGGACTGCAGTTGGTGGATGAATTGCGTTTGTGCATGTGTGAAAGTGGGGTGTGAATCCGCCAGTATAGGCCTGAGTGCTAACACCAGTTAGGAGGCATACCTCAGTGCCCTGAAGACCTAGGTCCAGCTTCCTAACCTGCTGGCCATAAGACCCTCGTAGCGGCACACATTGGCCATCTCTCCCAAACAACTGCATACCCGTTACTCTTCCAAAGTCCCCTCTTCTGCCATATGTTACATATTAAACCGATAGGAATATCTTTTTTCCCACTCCAAACGACAAGGCATGCATGCCGTTTTAATGTATCGAAAACTCCTCACCCCATTGTCACCTTTGAATTTCTCCCACATTTGCCCAAAAACGTTTATAAGGCACACACAACCTTCTCTTCTCCGTTAAAATGAGAAAATATGTCACAAACCCAGAGCATTGTGTATTTATTCTGCTGCTGCTATAGAATGATTCCTTATGTTATGAGCGAATAATTAACTTTTCTTCCTTTTAAAAAACAAAGGGCCGGGGACGGTGGCTCATGCCTGTAATCCCAGCATTTTGGGAGGCCCATGCTGGAGGATCATCTGAGGTCAGGAGTTCGAGACCAGCCTGCCTAATATGGCGAAACCCCGTCTCTACTAAAAATACAAAAATCAGCCGGGTGTGGTGGCACGTGCCTGTAGTCCCAGCTACTCGGGAGGCTGAGGCAGGAGAATCGCTTGAACCTAGGAGACGGGGGTGGGGGGGTTGCAGTGAGCCAAGATGGCACCACTGCACGCCAGGCTGGGTGACAAAGCGAGACTCCCTCAAAAAAAAAAAAAAAAAACACACACACACACACACATAAAGGTGTGAACCAAAGAGATCCAAGAAGTGAAAGGCAGAGTCCTGGCCTCAGAGCTGTGCAAGGCTCTTACTCTCCAACTTAATGGATGACGAGCCCTTTTAAATGCGAGATGGCCTGTCATTAGGCCTTCAGAACTTTAAAACATGCAAGAATTGTGGCTCTATCTAGGTATCCATAGAAAAAGAGAAGGAAGAGGCTGGGTGTGGTGGCTCACCCGTGTAATCCCAGCACTTCGGGAGGCCAAGGCAGGCGGATCACCTGAGGTCAGGCATTCGAGACCAGCCTGGCCAACATAGTGAAACCCTGTCCCTACTAAAACTACAAATATTAGCCAGGCGTGGTGGGGGGCGCCTGTAGTCCCAGCTACTCGGGAGGCTGAGGCAGGAGAATCATTTAAACACAGGAGGTTAAGGTTGCAGTGAGCCTGGGCGACAGAGCAAGACTCCATCTCAAAAAAAAAAAAGAAAAGAAAAAAAGAGGAGGAAGATCCACCACCCATGATCTGCTGGAAAGGGGCAGGTGGCAGGACTGTGCGCCACCTGCCCTCAGCCTAAGGGACTGTGACAAGGGACTAGAAAGCTCTTAGACTTTCTAGTCTAAGACTCTGGACTCTGGCTCGTGGGTGCCATGACAGGTGGCCGCTCCTCCCCAAAACCTGCCTTTCGGTGCCCATGGTCTGCAGCTCCAGGCTCTCGACCCCAGGCAGCCAGGTTGGCATAAAGACAGCAGTACCAGCCCTGAAATGTGGCCCTGAAGCCAGCCAGGAAGGCCAGGGAAAGAGTGTGGCCCGATGACTCATCTTTGCCTTAGGACAACTGCTGGTGTGAGGAAACCAGTGGGCTCTAAAAGCCCCGGCCTGCTGGTATTCTGGGGGGTCAGTGGCCCATGGACCCTCTTAGCCACCAGCTCTGAGCTGGCCCAGGGCCAAGAACACAGCCACCACCTTTGGCCACCCCAGGAGTGCCAAGAGTAAAACTGTCACTGTGGTTCCAGGGAGTCTTTGGGCCACAGAGGGTGTTATTACTGGAACAGAAATGTAGACAGAGGGGTATGGATTGCCATAAAAGTGTCTACCATATAAACAACAGTAAAGTCACACACACACACACACACACACACACACACACACACACACACACAATGTGGGGGGTGGGGGAGAGAGAAAAATCAAAACATCTAAAAATAAACAATGTTCAGAAAAAAGAGGTTTTCAAAAAGGAAGTGGAAACTTGGTCTTTGTGGGTGGTTTTCCTCTCCTCCCAGTTGTCCCCACCGACCCGTCACTGCCTCCCCCAACCAGCCCTAATCACTGTAGGCTCAACTTTAACCAAAGGACTACCTCATTAT'
ref = 'GGTCTGTCCGTCATGCTGAGGGGTTATTTTTATTTCCCGGCTCTCGGGCTGTTACTGAGTTGCCAGGACCTTATCAAAGCGCAGCCGGCTCCAGCCGACTCCCCCGGGCCAGGCTCGCGGAGCCAGTGATCCCGCCGCGCCAATCACAGCCGCGCTCGGCCGGCCCGCGCCGCCCGCCTTAAAGGGACAGCGCCCCGCGCCCGCCCCGCCCCGCCCCGGCCCGCCCCCGCCGCACGCTCCGCTCCCGCGTTAACCCTTCCCCGGCCTCTCCTCCGCCCCCTGCCAGCCCCCCGCGGCCCCGAGGGCATTCTCAGCCCGAGTTCTGATTTCTTCTTTTAAGTTCTCCGGCCTCTTTGCTCCCGCCTCCTCCCCCTGCCTGCCCTCGCTCTCCCCGGCTCTGGAATGCAACAGTTTCTGGTAGCATTATGGCCATCGCAAGAACTGCAAGCAAGCAGATTTTTTTTTTTCTATCATCAAACGCTTTTCTACCTCTATTCAAGTTTCCCTGGGAATTTGGGCTTGGAGGACTTTATAAAGGGAGTTCCGGCCCTGGGGTTGGCTGGCCCGCTGTCATAGGGCGGACTCGGGCCCAGCCGCTGGAGGCGTGTGCATGCCTGTGCGCGCTTGTGCACGTGCATCTGAGTTCGGGTAAATATAGGACTGCAGTTGGTGGATGAATTGCGTTTGTGCATGTGTGAAAGTGGGGTGTGAATCCGCCAGTATAGGCCTGAGTGCTAACACCAGTTAGGAGGCATACCTCAGTGCCCTGAAGACCTAGGTCCAGCTTCCTAACCTGCTGGCCATAAGACCCTCGTAGCGGCACACATTGGCCATCTCTCCCAAACAACTGCATACCCGTTACTCTTCCAAAGTCCCCTCTTCTGCCATATGTTACATATTAAACCGATAGGAATATCTTTTTTCCCACTCCAAACGACAAGGCATGCATGCCGTTTTAATGTATCGAAAACTCCTCACCCCATTGTCACCTTTGAATTTCTCCCACATTTGCCCAAAAACGTTTATAAGGCACACACAACCTTCTCTTCTCCGTTAAAATGAGAAAATATGTCACAAACCCAGAGCATTGTGTATTTATTCTGCTGCTGCTATAGAATGATTCCTTATGTTATGAGCGAATAATTAACTTTTCTTCCTTTTAAAAAACAAAGGGCCGGGGACGGTGGCTCATGCCTGTAATCCCAGCATTTTGGGAGGCCCATGCTGGAGGATCATCTGAGGTCAGGAGTTCGAGACCAGCCTGCCTAATATGGCGAAACCCCGTCTCTACTAAAAATACAAAAATCAGCCGGGTGTGGTGGCACGTGCCTGTAGTCCCAGCTACTCGGGAGGCTGAGGCAGGAGAATCGCTTGAACCTAGGAGACGGGGGTGGGGGGGTTGCAGTGAGCCAAGATGGCACCACTGCACGCCAGGCTGGGTGACAAAGCGAGACTCCCTCAAAAAAAAAAAAAAAAAACACACACACACACACACATAAAGGTGTGAACCAAAGAGATCCAAGAAGTGAAAGGCAGAGTCCTGGCCTCAGAGCTGTGCAAGGCTCTTACTCTCCAACTTAATGGATGACGAGCCCTTTTAAATGCGAGATGGCCTGTCATTAGGCCTTCAGAACTTTAAAACATGCAAGAATTGTGGCTCTATCTAGGTATCCATAGAAAAAGAGAAGGAAGAGGCTGGGTGTGGTGGCTCACCCGTGTAATCCCAGCACTTCGGGAGGCCAAGGCAGGCGGATCACCTGAGGTCAGGCATTCGAGACCAGCCTGGCCAACATAGTGAAACCCTGTCCCTACTAAAACTACAAATATTAGCCAGGCGTGGTGGGGGGCGCCTGTAGTCCCAGCTACTCGGGAGGCTGAGGCAGGAGAATCATTTAAACACAGGAGGTTAAGGTTGCAGTGAGCCTGGGCGACAGAGCAAGACTCCATCTCAAAAAAAAAAAAGAAAAGAAAAAAAGAGGAGGAAGATCCACCACCCATGATCTGCTGGAAAGGGGCAGGTGGCAGGACTGTGCGCCACCTGCCCTCAGCCTAAGGGACTGTGACAAGGGACTAGAAAGCTCTTAGACTTTCTAGTCTAAGACTCTGGACTCTGGCTCGTGGGTGCCATGACAGGTGGCCGCTCCTCCCCAAAACCTGCCTTTCGGTGCCCATGGTCTGCAGCTCCAGGCTCTCGACCCCAGGCAGCCAGGTTGGCATAAAGACAGCAGTACCAGCCCTGAAATGTGGCCCTGAAGCCAGCCAGGAAGGCCAGGGAAAGAGTGTGGCCCGATGACTCATCTTTGCCTTAGGACAACTGCTGGTGTGAGGAAACCAGTGGGCTCTAAAAGCCCCGGCCTGCTGGTATTCTGGGGGGTCAGTGGCCCATGGACCCTCTTAGCCACCAGCTCTGAGCTGGCCCAGGGCCAAGAACACAGCCACCACCTTTGGCCACCCCAGGAGTGCCAAGAGTAAAACTGTCACTGTGGTTCCAGGGAGTCTTTGGGCCACAGAGGGTGTTATTACTGGAACAGAAATGTAGACAGAGGGGTATGGATTGCCATAAAAGTGTCTACCATATAAACAACAGTAAAGTCACACACACACACACACACACACACACACACACACACACACACACAATGTGGGGGGTGGGGGAGAGAGAAAAATCAAAACATCTAAAATCAAAATAAACAATGTTCAGAAAAAAGAGGTTTTCAAAAAGGAAGTGGAAACTTGGTCTTTGTGGGTGGTTTTCCTCTCCTCCCAGTTGTCCCCACCGACCCGTCACTGCCTCCCCCAACCAGCCCTAATCACTGTAGGCTCAACTTTAACCAAAGGACTACCTCATTAT'
print(nchar(alt))
print(nchar(ref))

[1] 2846
[1] 2851


## Using 10 independent positional input seq encodings

In [9]:
%%R
# First load enformer predictions and the seqeunce predictions were derived from

files = list.files('/home/kribelba/updepla/users/kribelba/RMCE_results/Enformer_Axin2_pred/pred_Enformer/2024_10_21/',full.names=T)

#files = list.files('path_to_prediction_files')
pred57 = read.csv(files[grep('688_lib_5p7',files)],header=T)
lab57 = read.csv(files[grep('seqs_lib_5p7',files)],header=T)
print(tail(pred57))
print(tail(lab57[,1:2]))


          X        X0         X1         X2        X3
3843835 890 0.1114654 0.04577652 0.04156170 0.3506854
3843836 891 0.1559188 0.06310277 0.10773227 0.4520571
3843837 892 0.1542170 0.07039242 0.04286680 0.5037035
3843838 893 0.1614316 0.07651050 0.43545747 0.5424047
3843839 894 0.1480612 0.06093595 0.06308208 0.4249329
3843840 895 0.1102044 0.04231025 0.05482549 0.3654277
        X                                                               id
4285 1424             CGislandDel-AHR_TF65_AHR_shiftbin_59_bc_TAGCTTAGTTAG
4286 1425              CGislandDel-AHR_AHR_AHR_shiftbin_59_bc_TAGCTTAGTTAG
4287 1426           CGislandDel-IRF2_IRF2_IRF2_shiftbin_59_bc_TAGCTTAGTTAG
4288 1427       CGislandDel-MF2A_TF65_FX1_TF65_shiftbin_59_bc_TAGCTTAGTTAG
4289 1428  CGislandDel-MEF2A_PU1_FX1_BCH1_TF65_shiftbin_59_bc_TAGCTTAGTTAG
4290 1429 CGislandDel-MEF2A_PU1_FX1_BCH1_MEF2A_shiftbin_59_bc_TAGCTTAGTTAG


In [10]:
%%R

# annotate predictions with positional encoding and BCs, etc
annotate_pred = function(pred,lab){
    # this takes the predictions (pred) and seqeunces with labels (lab) from above and adds labels to the prediciton files

    predn=pred
    colnames(predn) = c('bin','DNAse12','DNAse69','CAGE','H3K27ac') #rename column base on the enformer heads used for predictions
    nbins = dim(predn)[1]/length(lab[,1]) # how many unique prediciton bins for one particular sequence shift
        
    predn$id = unlist(lapply(lab$id,function(x){rep(x,nbins)})) # ad label to each prediction bin
    predn$geno = sapply(predn$id,function(x){
        s=strsplit(x,split="_")[[1]]
        return(paste(s[1:(length(s)-4)],collapse="_"))}) # add genotype to each pred bin
    predn$BC =sapply(predn$id,function(x){
        s=strsplit(x,split="_")[[1]]
        return(s[(length(s))])})     # add BC info
    predn$shiftpos =sapply(predn$id,function(x){
        s=strsplit(x,split="_")[[1]]
        return(s[(length(s)-2)])})   # add pred shift position (10 shifts total)
    predn$geno_pos = paste0(predn$geno,"_",predn$shiftpos)
    return(predn)
}

In [11]:
%%R
# now annotate
pred57=annotate_pred(pred57,lab57)
head(pred57)


  bin    DNAse12    DNAse69       CAGE   H3K27ac
1   0 0.12998750 0.04128101 0.04474570 0.4951257
2   1 0.08336915 0.02329321 0.06095159 0.2486073
3   2 0.01844230 0.01896473 0.02674513 0.2288125
4   3 0.02245893 0.02418058 0.03271722 0.3372698
5   4 0.04622030 0.01923825 0.05697825 0.1927668
6   5 0.06100839 0.01465900 0.04428573 0.2032412
                                id geno           BC shiftpos geno_pos
1 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121
2 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121
3 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121
4 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121
5 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121
6 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121


In [13]:
%%R

# step 1: find the right bin

total_bins = (393216/128) # input lenght divided by bin lenght
print(paste0('total_bins = ',total_bins))

outbin = sum(pred57$id==unique(pred57$id)[1]) # number of bins for which predictions exist
print(paste0('bins with predictions = ',outbin))
outlength = outbin*128 # total sequence length for which predicition exists = outbin * bin length
print(paste0('seq length with predictions = ',outlength))

empty_bins = (total_bins-outbin)/2 # the number of flanking seq bins for which no prediciton exists
print(paste0('bins without predictions = ',empty_bins))
#1088.0 # 1088 bins are used as 'padding'
empty_seqlen = 1088*128 # seq length of those empty bins on each side
print(paste0('seq length without predictions = ',empty_seqlen))


[1] "total_bins = 3072"
[1] "bins with predictions = 896"
[1] "seq length with predictions = 114688"
[1] "bins without predictions = 1088"
[1] "seq length without predictions = 139264"


In [15]:
%%R
pred_seq57 = subseq(DNAStringSet(lab57[1:(dim(lab57)[1]/3),3]),empty_seqlen+1,empty_seqlen+outlength) # the actual sequence for which predicitons exists
names(pred_seq57) = lab57[1:(dim(lab57)[1]/3),2]


In [16]:
%%R
head(pred_seq57)

DNAStringSet object of length 6:
     width seq                                              names               
[1] 114688 CCCAGTTGTTTTAAGAATGTATA...TGAGGCACCACGCCTGGCCCGC ref_shiftbin_121_...
[2] 114688 CCCAGTTGTTTTAAGAATGTATA...CACCACGCCTGGCCCGCACTCT alt_shiftbin_121_...
[3] 114688 CCCAGTTGTTTTAAGAATGTATA...GTTAGGAGACAGCACTTCTCCT wt-FX1_DBP_FX1_sh...
[4] 114688 CCCAGTTGTTTTAAGAATGTATA...TCCTATTTAGAACCCTCTTATG wt-CTCF_in_shiftb...
[5] 114688 CCCAGTTGTTTTAAGAATGTATA...TCCTATTTAGAACCCTCTTATG wt-CTCF_out_shift...
[6] 114688 CCCAGTTGTTTTAAGAATGTATA...CTCTCAGGTTAGGAGACAGCAC wt-FX1_BATF_FX1_s...


In [18]:
%%R
# split into seqeunces that have synthetic fragements and ref/alt which is the endogenous enhancer
pred_seq57nra= pred_seq57[setdiff(1:length(pred_seq57),unlist(lapply(c('ref','alt'),function(x){grep(x,names(pred_seq57))})))]
pred_seq57ra= pred_seq57[unlist(lapply(c('ref','alt'),function(x){grep(x,names(pred_seq57))}))]


In [27]:
# define patterns for UTR and enhancer
down_DPE = 'CAAAGCGCAGCCGGCTCCAGC' #This sequence is downstream of the DPE motif right in the 5'UTR
ibtw_DPEINR = 'CCCCCGGGCCAG'
ibtw_DPEINR_alt='GCTCGCGGAGCC'
ri_bin = 'ATGTTCAGAAAAA' # this seq is upstream of the TFBS modification within the native AXIN2 enhancer
le_bin = 'GAAAAATCAA' # this seq is upstream of the TFBS modification within the native AXIN2 enhancer
art_enh_bef = 'TGCCCATGGTCT'
art_enh_bef2= 'GCAGCTCCAGGC'


In [30]:
%%R

get_bin_alt = function(dnaseq,string,altstring){
    # this function finds the bin in which a particular sequence is found
    # if seq-pattern is split by a bin, input a modified pattern
    vp=Views(DNAString(as.character(dnaseq)),start = seq(1,width(dnaseq),128),end = seq(128,width(dnaseq)+1,128))
    g=grep(string,as.character(vp))
    ga=grep(altstring,as.character(vp)) 
    if (length(g)>0){gg=g}
    if (length(g)==0){gg=ga}
    return(gg-1) # -1 for python indexing
}

# functions to identify bin 
annotate_bins_alt = function(pred,predseq,utrseq,utrseq_alt,enhseq,enhseq_alt,ext_bin_UTR,ext_bin_enh){
    # this function annotates the prediction file to indicate which of the bins represent the UTR (expression) or the local enhancer)
    # it has two options in case the seqeunce that is being searched falls inbetween two bins
    utr_bin = sapply(1:length(predseq),function(x){get_bin_alt(predseq[x],utrseq,utrseq_alt)})
    enh_bin = sapply(1:length(predseq),function(x){get_bin_alt(predseq[x],enhseq,enhseq_alt)})
    bindf = data.frame(id = names(predseq),bin_DPE = utr_bin,binBS =enh_bin )
    bindf$geno = unlist(lapply(bindf$id,function(x){i=strsplit(x,split="_")[[1]]
                                                     paste(i[c(1:(length(i)-4),(length(i)-2))],collapse="_")}))
    predn=pred
    predn$BSbin = bindf$binBS[match(predn$geno_pos,bindf$geno)]
    predn$is_BSbin = predn$bin == predn$BSbin
    predn$is_extBSbin = abs(predn$bin-predn$BSbin) <= ext_bin_enh
    predn$UTRbin = bindf$bin_DPE[match(predn$geno_pos,bindf$geno)]
    predn$is_UTRbin = predn$bin == predn$UTRbin
    predn$is_extUTRbin = abs(predn$bin-predn$UTRbin) <= ext_bin_UTR
    #predn$geno_bin = paste0(predn$geno,'_',predn$bin)
    return(predn)}

annotate_bins_twoways_alt = function(pred,predseq,utrseq,utrseq_alt,enhseq,enhseq_alt,ext_bin_UTR,ext_bin_enh){
    # this function annotates the prediction file to indicate which of the bins represent the UTR (expression) or the local enhancer)
    # it has two options in case the seqeunce that is being searched falls inbetween two bins
    # this one is allows a more flexible binning for the synthetic enhancer region
    utr_bin = sapply(1:length(predseq),function(x){get_bin_alt(predseq[x],utrseq,utrseq_alt)})
    enh_bin = sapply(1:length(predseq),function(x){get_bin_alt(predseq[x],enhseq,enhseq_alt)})
    bindf = data.frame(id = names(predseq),bin_DPE = utr_bin,binBS =enh_bin )
    bindf$geno = unlist(lapply(bindf$id,function(x){i=strsplit(x,split="_")[[1]]
                                                     paste(i[c(1:(length(i)-4),(length(i)-2))],collapse="_")}))
    predn=pred
    predn$BSbin = bindf$binBS[match(predn$geno_pos,bindf$geno)]
    predn$is_BSbin = predn$bin == predn$BSbin
    predn$is_extBSbin = (predn$bin-predn$BSbin) %in% ext_bin_enh
    predn$UTRbin = bindf$bin_DPE[match(predn$geno_pos,bindf$geno)]
    predn$is_UTRbin = predn$bin == predn$UTRbin
    predn$is_extUTRbin = (predn$bin-predn$UTRbin) %in% ext_bin_UTR
    #predn$geno_bin = paste0(predn$geno,'_',predn$bin)
    return(predn)}


In [35]:
%%R
#anotate
# split into predictions that have synthetic fragements and ref/alt which is the endogenous enhancer

pred57sra = pred57[pred57$geno %in% c('ref','alt'),]
pred57sart = pred57[!(pred57$geno %in% c('ref','alt')),]
print(dim(pred57sra))
print(dim(pred57sart))

down_DPE = 'CAAAGCGCAGCCGGCTCCAGC' #This sequence is downstream of the DPE motif right in the 5'UTR
ibtw_DPEINR = 'CCCCCGGGCCAG'
ibtw_DPEINR_alt='GCTCGCGGAGCC'
ri_bin = 'ATGTTCAGAAAAA' # this seq is upstream of the TFBS modification within the native AXIN2 enhancer
le_bin = 'GAAAAATCAA' # this seq is upstream of the TFBS modification within the native AXIN2 enhancer
art_enh_bef = 'TGCCCATGGTCT'
art_enh_bef2= 'GCAGCTCCAGGC'

#annotate ref/alt and the synthetic enahancers
pred57ra = annotate_bins_alt(pred57sra,pred_seq57ra,ibtw_DPEINR,ibtw_DPEINR_alt,le_bin,ri_bin,1,3)                                                
pred57art = annotate_bins_twoways_alt(pred57sart,pred_seq57nra,ibtw_DPEINR,ibtw_DPEINR_alt,art_enh_bef,art_enh_bef2,seq(-1,1,1),seq(-2,4))                                                

pred57a = rbind(pred57ra,pred57art)
print(head(pred57a))

print(table(pred57a$BSbin)) # there are several locations where the "enhancer bins" are located, since the distance to the syn and endogenous enhancers vary in length and the genotypes containgin the GCdel promoter also shorten the enhancer to promoter distance
print(table(pred57a$UTRbin)) # but only one UTR bin, since the shifts were all done within <128bp, therefore not necessarily affecting the bin location of the UTR, which is further downstream of the enhancer


[1] 53760    10
[1] 3790080      10
  bin    DNAse12    DNAse69       CAGE   H3K27ac
1   0 0.12998750 0.04128101 0.04474570 0.4951257
2   1 0.08336915 0.02329321 0.06095159 0.2486073
3   2 0.01844230 0.01896473 0.02674513 0.2288125
4   3 0.02245893 0.02418058 0.03271722 0.3372698
5   4 0.04622030 0.01923825 0.05697825 0.1927668
6   5 0.06100839 0.01465900 0.04428573 0.2032412
                                id geno           BC shiftpos geno_pos BSbin
1 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121   495
2 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121   495
3 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121   495
4 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121   495
5 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121   495
6 ref_shiftbin_121_bc_AGACCGGTCGTA  ref AGACCGGTCGTA      121  ref_121   495
  is_BSbin is_extBSbin UTRbin is_UTRbin is_extUTRbin
1    FALSE       FALSE    475

In [38]:
%%R
## compute averages

XDF = pred57a
avg_pred_dfs_UTR = do.call(cbind,lapply(c('CAGE','DNAse69','H3K27ac'),function(x){data.frame(data.frame(XDF %>% 
                                                                         filter(is_extUTRbin == "TRUE") %>% 
                                                                         group_by(id,geno) %>% 
                                                                                    summarize(logsum = log2(sum(.data[[x]], na.rm = TRUE)), .groups='drop') )
                                                              %>% group_by(geno) %>% summarize(avg = mean(logsum,na.rm=T)))}))[,c(1,2,4,6)]
colnames(avg_pred_dfs_UTR) = c('geno',paste0('agg_pred_UTR_',c('CAGE','DNAse69','H3K27ac')))

sd_pred_dfs_UTR = do.call(cbind,lapply(c('CAGE','DNAse69','H3K27ac'),function(x){data.frame(data.frame(XDF %>% 
                                                                         filter(is_extUTRbin == "TRUE") %>% 
                                                                         group_by(id,geno) %>% 
                                                                                    summarize(logsum = log2(sum(.data[[x]], na.rm = TRUE)), .groups='drop') )
                                                              %>% group_by(geno) %>% summarize(avg = sd(logsum,na.rm=T)))}))[,c(1,2,4,6)]

colnames(sd_pred_dfs_UTR) = c('geno',paste0('agg_pred_SD_UTR_',c('CAGE','DNAse69','H3K27ac')))

avg_pred_dfs_enh = do.call(cbind,lapply(c('CAGE','DNAse69','H3K27ac'),function(x){data.frame(data.frame(XDF %>% 
                                                                         filter(is_extBSbin == "TRUE") %>% 
                                                                         group_by(id,geno) %>% 
                                                                                    summarize(logsum = log2(sum(.data[[x]], na.rm = TRUE)), .groups='drop') )
                                                              %>% group_by(geno) %>% summarize(avg = mean(logsum,na.rm=T)))}))[,c(1,2,4,6)]
colnames(avg_pred_dfs_enh) = c('geno',paste0('agg_pred_enh_',c('CAGE','DNAse69','H3K27ac')))

sd_pred_dfs_enh = do.call(cbind,lapply(c('CAGE','DNAse69','H3K27ac'),function(x){data.frame(data.frame(XDF %>% 
                                                                         filter(is_extBSbin == "TRUE") %>% 
                                                                         group_by(id,geno) %>% 
                                                                                    summarize(logsum = log2(sum(.data[[x]], na.rm = TRUE)), .groups='drop') )
                                                              %>% group_by(geno) %>% summarize(avg = sd(logsum,na.rm=T)))}))[,c(1,2,4,6)]
colnames(sd_pred_dfs_enh) = c('geno',paste0('agg_pred_SD_enh_',c('CAGE','DNAse69','H3K27ac')))

print(head(avg_pred_dfs_UTR))
print(head(avg_pred_dfs_enh))

print(head(avg_pred_dfs_UTR[avg_pred_dfs_UTR$geno %in% c('ref','alt'),]))



                       geno agg_pred_UTR_CAGE agg_pred_UTR_DNAse69
1   CGislandDel-AHR_AHR_AHR          1.363139            0.1280745
2  CGislandDel-AHR_PU1_TF65          2.227390            0.5076635
3  CGislandDel-AHR_TF65_AHR          1.396349            0.1288258
4         CGislandDel-BACH1          1.416735            0.1515100
5          CGislandDel-BATF          1.479889            0.1670778
6 CGislandDel-BCH1_PU1_BCH1          3.647366            1.1961827
  agg_pred_UTR_H3K27ac
1             3.600427
2             4.369290
3             3.545560
4             3.533661
5             3.653272
6             5.496392
                       geno agg_pred_enh_CAGE agg_pred_enh_DNAse69
1   CGislandDel-AHR_AHR_AHR       -0.76039348            -1.863607
2  CGislandDel-AHR_PU1_TF65        1.63367143             2.590776
3  CGislandDel-AHR_TF65_AHR       -0.57044825            -1.699803
4         CGislandDel-BACH1       -0.66893239            -1.769667
5          CGislandDel-BATF        

In [42]:
%%R
# save to file
enh = merge(avg_pred_dfs_enh,sd_pred_dfs_enh,by='geno')
UTR = merge(avg_pred_dfs_UTR,sd_pred_dfs_UTR,by='geno')
avgsd_df= merge(UTR,enh,by='geno')
print(head(avgsd_df))

#write.csv(avgsd_df,'/outpath/SUM_PRED_5p7_logsumacross_bins_avg_across_3BCs_and_10POS_CAGE_DNAS269_K27ac_3binsUTR_7bins_enh_with_SD_pred_of_2024_10_21.csv',row.names=F)


                       geno agg_pred_UTR_CAGE agg_pred_UTR_DNAse69
1   CGislandDel-AHR_AHR_AHR          1.363139            0.1280745
2  CGislandDel-AHR_PU1_TF65          2.227390            0.5076635
3  CGislandDel-AHR_TF65_AHR          1.396349            0.1288258
4         CGislandDel-BACH1          1.416735            0.1515100
5          CGislandDel-BATF          1.479889            0.1670778
6 CGislandDel-BCH1_PU1_BCH1          3.647366            1.1961827
  agg_pred_UTR_H3K27ac agg_pred_SD_UTR_CAGE agg_pred_SD_UTR_DNAse69
1             3.600427            0.9996123               0.2086080
2             4.369290            1.0637288               0.1865373
3             3.545560            1.0415712               0.2038824
4             3.533661            0.9931093               0.2392857
5             3.653272            1.0271885               0.2654551
6             5.496392            0.8393532               0.1882620
  agg_pred_SD_UTR_H3K27ac agg_pred_enh_CAGE agg_pred_en